<a href="https://colab.research.google.com/github/RodolfoFigueroa/madi2022-1/blob/main/4_Divide_y_venceras_con_estructuras_de_datos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [105]:
import numpy as np

En esta sesión, veremos dos estructuras de datos que están muy ligadas a las ideas de los algoritmos de divide y vencerás, que suelen ser muy útiles para realizar operaciones que suelen requerirse bastante.

# Trie

Consideremos el siguiente problema. Dadas dos palabras, decimos que su "similitud" es el mayor número de letras iniciales en las que coinciden, es decir, es el tamaño del prefijo común más grande entre ambas palabras. Dado un conjunto de palabras $C$, ,y una nueva palabra, $s$, queremos saber la mayor similitud posible con alguna letra del conjunto. 

¿Cómo proceder? Una forma de hacerlo es haciendo una búsqueda exhaustiva, donde para cada palabra de $C$ la comparamos con la palabra $s$, y vamos verificando letra por letra los prefijos hasta que dejen de coincidir. Podemos acotar la complejidad en tiempo por $O(|C|\cdot |s|)$. ¿Qué pasa si queremos repetir el proceso para una cantidad muy grande de palabras s?

Otra forma de atacar el problema sería ordenar las palabras en orden alfabético, incluyendo a $s$, y posteriormente comparar a $s$ con las palabras vecinas en el ordenamiento (queda como ejercicio probar que en efecto uno de sus vecinos es la palabra que cumple tener mayor similitud con $s$). ¿Qué complejidad tiene esto?

La forma que veremos introduce una estructura, que resulta ser muy poderosa para este tipo de problemas, es un árbol especial, que tiene por nombre *trie*. Un trie es un árbol donde cada nodo puede tener $26$ hijos (uno por cada letra del abecedario). De este modo, si una palabra tiene las letras "xy" de forma consecutiva, el respectivo nodo tendrá un hijo no nulo en la entrada correspondiente a "y". Veamos un ejemplo de esta estructura.

Consideremos las palabras `[abcdef, abecede, hola, hoja, puedes, luz]`. El trie respectivo a este conjunto de palabras es el siguiente:

```
       (Nodo raíz)
      /   |   |  \
     a    h   p   l
     |    |   \    \
     b    o    u    u
    / \   | \   \    \
   c   e  l  j   e    z
   |   |  |  |   |
   d   c  a  a   d
   |   |         |
   e   e         e
   |   |         |
   f   d         s
       |
       e
```

Esta estructura nos permite identificar de manera rápida y sencilla el mayor prefijo que coincide con cierta palabra. Veamos cómo implementar esta estructura y cómo usarla para nuestro problema.

In [12]:
def char_index(c):
    return ord(c) - ord('a')

class TrieNode:
    def __init__(self):
        self.children = [None]*26
        
    def insert(self, word):
        word = word.casefold()
        l = len(word)
        curr = self
        for c in word:
            idx = char_index(c)
            if curr.children[idx] is None:
                curr.children[idx] = TrieNode()
            curr = curr.children[idx]
            
    def count_prefix(self, word):
        curr = self
        count = 0
        for c in word:
            idx = char_index(c)
            if not curr.children[idx]:
                return count
            else:
                curr = curr.children[idx]
                count += 1
        return count


C = ["abcdef", "abecede", "hola", "hoja", "puedes", "luz"]
root = TrieNode()
for c in C:
    root.insert(c)

S = ["abaco", "holandes", "paz"]
for s in S:
    print(root.count_prefix(s))

2
4
1


# Segmentos

Dada una lista $L$ de números reales ($|L| = n$), queremos saber la suma de los elementos en un intervalo dado, es decir $L_i + L_{i+1} + \dots + L_j$ para determinados $i \leq j$. Además de esto, queremos poder actualizar nuestra lista $L$. Es decir, tenemos dos tipos de operaciones:

*   Preguntar por la suma de los valores con índices en el intervalo $[i,j]$.
*   Actualizar el valor de cierto elemento $L_i$.

Una forma sencilla de poder recuperar el valor de la suma en cierto intervalo es guardando una lista de sumas acumuladas, sin embargo, esto hace que las operaciones de actualización de elementos sean costosas.

¿Qué pasa si dividimos la lista en sublistas de longitud $k$? (Veremos después qué $k$ nos conviene). Tendremos un total de $\left\lceil\frac{n}{k}\right\rceil$ 'bloques', para cada bloque guardaremos el valor de la suma de sus elementos. ¿Qué tan costosas son las operaciones de actualización y preguntar por sumas? 

* Para actualizar, únicamente debemos actualizar el valor del elemento, así como la suma correspondiente en el bloque al que pertenece, por lo que cada actualización nos toma $O(1)$. 
* Para calcular la suma de un intervalo $I$, notamos que para cada bloque $B$ existen tres casos:

    1. $B\cap I=\emptyset$. En este caso, no hacemos nada. ($O(1)$)
    2. $B\subseteq I$. Aquí, simplemente añadimos la suma del bloque entero (que tenemos guardada). ($O(1)$)
    3. $B\cap I \neq \emptyset \wedge B\not\subseteq I$. En este caso, tenemos que iterar sobre $B$ para encontrar cuáles son los elementos que están en $I$, y sumarlos. ($O(k)$)
    
El tercer caso sólo puede pasar para dos bloques: los que están en el extremo izquierdo y derecho del intervalo. Por lo tanto, la complejidad de estos casos es $O(k)$. 

Por otro lado, como hay un total de $\frac{n}{k}$ bloques, el primer y segundo caso no pueden pasar más de esta cantidad, así que su complejidad es $(O(\frac{n}{k}))$.

Por lo tanto, la complejidad del problema completo es $O\left(\frac{n}{k}+k\right)$.

Entonces, queremos encontrar la $k$ que minimice esta cantidad. Derivando e igualando a cero, obtenemos $k=\sqrt{n}$.

## Segment tree

Una manera de representar esta estructura es utilizando un árbol binario, donde cada nodo representa a un intervalo, y tiene los siguientes atributos:

* `value`: Suma del intervalo.
* `left`: Hijo izquierdo.
* `right`: Hijo derecho.
* `start`: Extremo izquierdo del intervalo.
* `end`: Extremo derecho del intervalo.

El intervalo de un nodo padre es la unión de los intervalos de sus dos hijos. Los nodos hoja corresponden a los intervalos triviales de la lista original (i.e. $[L_0, L_0], [L_1, L_1]$, etc.)

De este modo, logramos que preguntar por la suma en un intervalo $[l,r]$ sea $O(logn)$, mientras que actualizar también nos tomará $O(logn)$, ya que esta es la altura de nuestro árbol.

Veamos una implementación de esta estructura y un ejemplo de su uso.

In [132]:
class SegNode:
    def __init__(self, L, l=None, r=None):
        self.value = 0
        
        if l is None:
            l = 0
        if r is None:
            r = len(L) - 1
        
        self.start = l
        self.end = r
        if self.start == self.end:
            self.value = L[self.start]
        else:
            m = (l+r)//2
            self.left = SegNode(L, l=self.start, r=m)
            self.right = SegNode(L, l=m+1, r=self.end)
            if self.left is not None:
                self.value += self.left.value
            if self.right is not None:
                self.value += self.right.value
    
    def _arr_str(self):
        return f'[{self.start}, {self.end}]'
    
    def __repr__(self):
        return f'{self._arr_str()}\nHijos: {self.left._arr_str()}, {self.right._arr_str()}'
    
    def query(self, l=None, r=None):
        if l is None:
            l = self.start
        if r is None:
            r = self.end
            
        if l <= self.start and self.end <= r:
            return self.value
        if self.end < l:
            return 0
        if r < self.start:
            return 0
        
        s = 0
        if self.left is not None:
            s += self.left.query(l, r)
        if self.right is not None:
            s += self.right.query(l, r)
        return s
    
    def update(self, i, val):
        if self.end < i or i < self.start:
            return self.value
        
        if self.end == self.start == i:
            self.value = val
        else:
            s = 0
            if self.left is not None:
                s += self.left.update(i, val)
            if self.right is not None:
                s += self.right.update(i, val)
        return self.value

In [133]:
L = np.random.randint(0, 30, 20)
print(L)
test = SegNode(L)

[ 5  5 24  2 13 21 19 28  0 17 29  6  1 11 12  0 10 23 17 24]
